# Few-Shots Prompting

Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

## References:
* [Touvron et al. 2023](https://arxiv.org/pdf/2302.13971.pdf): present few shot properties  when models were scaled to a sufficient size
* [Kaplan et al., 2020](https://arxiv.org/abs/2001.08361)
* [Brown et al. 2020](https://arxiv.org/abs/2005.14165)


In [2]:
from _pipeline import create_payload, model_req

#### (1) Adjust the inbounding  Prompt, simulating inbounding requests from users or other systems
JOB_DESCRIPTION = """
We are hiring a Data Scientist with expertise in Python, Machine Learning, and SQL.
The candidate must have at least 3 years of experience and a background in statistical modeling.
Experience with cloud platforms (AWS, GCP) is a plus.
"""

RESUME = """
John Doe is a Data Scientist with 4 years of experience. Skilled in Python, Machine Learning,
and SQL. He has worked on predictive modeling projects and statistical analysis.
Familiar with AWS but lacks GCP experience.
"""

#### (2) Adjust the Few-Shot Prompting Technique
FEW_SHOT = f"""
You are an AI-powered hiring assistant. Your task is to analyze job descriptions 
and compare them with candidate resumes using structured reasoning.

**Scoring Guidelines:**
- If the candidate is missing a **"required"** skill, subtract **15 points**.
- If the candidate is missing a **"preferred" (plus)** skill, subtract **5 points**.
- If all required skills match, assign **100%** minus deductions.

---

**Example 1:**  
**Job Description:** Data Scientist with Python, Machine Learning, SQL, and AWS/GCP.  
**Candidate Resume:** Python, Machine Learning, SQL, AWS.  
**Analysis:**  
✔ Hard Skills: Matched  
✔ Soft Skills: Not explicitly stated in the resume  
✔ Experience: Matches (3+ years required, candidate has 4)  
⚠ Certification: AWS present, but GCP missing (-5 points)  

**Match Score:** 95%  
**Recommendation:** Highly recommended. Candidate aligns with most skills; GCP experience would be beneficial.

---

**Example 2:**  
**Job Description:** Software Engineer with Java, Kubernetes, and Cloud Computing.  
**Candidate Resume:** Java, Kubernetes, AWS.  
**Analysis:**  
✔ Hard Skills: Matched  
✔ Cloud Skills: AWS present, but no mention of GCP  
✔ Experience: Meets the required 5 years  
⚠ Missing Cloud Computing certification (-15 points)  

**Match Score:** 85%  
**Recommendation:** Strong fit, but missing key certification.

---

Now analyze the following job description and resume step by step, and provide a match score.  
Only highlight key matches, gaps, and a final score.

**Job Description:**  
{JOB_DESCRIPTION}  

**Candidate Resume:**  
{RESUME}  

**At the end, provide a one-sentence hiring recommendation based on their strengths and gaps.**
"""

PROMPT = FEW_SHOT

#### (3) Configure the Model Request
payload = create_payload(target="ollama",
                         model="mistral",  
                         prompt=PROMPT, 
                         temperature=0.7,  
                         num_ctx=768,  # Optimized for speed
                         num_predict=250,  # Ensures concise responses
                         stream=True)  # Enable streaming for better performance

### YOU DON'T NEED TO CONFIGURE ANYTHING ELSE FROM THIS POINT
# Send out to the model
time, response = model_req(payload=payload)
print(response)
if time: print(f'Time taken: {time}s')

{'model': 'mistral', 'prompt': '\nYou are an AI-powered hiring assistant. Your task is to analyze job descriptions \nand compare them with candidate resumes using structured reasoning.\n\n**Scoring Guidelines:**\n- If the candidate is missing a **"required"** skill, subtract **15 points**.\n- If the candidate is missing a **"preferred" (plus)** skill, subtract **5 points**.\n- If all required skills match, assign **100%** minus deductions.\n\n---\n\n**Example 1:**  \n**Job Description:** Data Scientist with Python, Machine Learning, SQL, and AWS/GCP.  \n**Candidate Resume:** Python, Machine Learning, SQL, AWS.  \n**Analysis:**  \n✔ Hard Skills: Matched  \n✔ Soft Skills: Not explicitly stated in the resume  \n✔ Experience: Matches (3+ years required, candidate has 4)  \n⚠ Certification: AWS present, but GCP missing (-5 points)  \n\n**Match Score:** 95%  \n**Recommendation:** Highly recommended. Candidate aligns with most skills; GCP experience would be beneficial.\n\n---\n\n**Example 2: